Install Libraries

In [ ]:
!pip install langchain
!pip install unstructured libmagic python-magic

HuggingFace Setup

In [62]:
HuggingFaceApiKey = "WRITE YOUR KEY HERE"
headers = {"Authorization": HuggingFaceApiKey}

Imports

In [63]:
import numpy as np
from langchain.document_loaders import UnstructuredURLLoader
import requests
from langchain.text_splitter import RecursiveCharacterTextSplitter

Web Scrapping

In [64]:
loader = UnstructuredURLLoader(urls=[
    "https://www.indiatoday.in/india/story/farmers-protest-haryana-kheri-chopta-stone-pelting-leaders-detained-tear-gas-ground-report-2506378-2024-02-23"
])
data_ = loader.load()
data_ = str(data_)
data_

'[Document(page_content=\'Tear gas fired, stones thrown as farmers\\\' meet in Haryana turns violent | Ground report\\n\\nThousands of farmers gathered in Haryana\\\'s\\xa0Kheri Chopta to participate in a march towards the Khanauri border. The meeting turned violent after police fired tear gas and detained farmer leaders.\\n\\nListen to Story\\n\\nLive TV\\n\\nShare\\n\\nAdvertisement\\n\\nFarmers have been stationed at the Punjab-Haryana border as they march to Delhi to pressure the central government to fulfil their demands. (PTI photo)\\n\\nAnmol Nath Bali\\n\\nKheri Chopta,\\n\\nUPDATED: Feb 23, 2024 23:45 IST\\n\\nIn Short\\n\\nFarmers\\\' meeting in Haryana\\\'s Kheri Chopta turns violent\\n\\nPolice fire tear gas, detain farmer leaders\\n\\nStone pelting by farmers escalates tension\\n\\nA meeting of farmers in Haryana\\\'s Kheri Chopta turned violent on Friday as police fired tear gas and detained leaders. Thousands had gathered at the site after farmers\\\' unions and khap pan

Splitting The Data

In [65]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

chunks = splitter.split_text(data_)
chunks[0]

"[Document(page_content='Tear gas fired, stones thrown as farmers\\' meet in Haryana turns violent | Ground report\\n\\nThousands of farmers gathered in Haryana\\'s\\xa0Kheri Chopta to participate in a march towards the Khanauri border. The meeting turned violent after police fired tear gas and detained farmer leaders.\\n\\nListen to Story\\n\\nLive TV\\n\\nShare\\n\\nAdvertisement\\n\\nFarmers have been stationed at the Punjab-Haryana border as they march to Delhi to pressure the central government to fulfil their demands. (PTI photo)\\n\\nAnmol Nath Bali\\n\\nKheri Chopta,\\n\\nUPDATED: Feb 23, 2024 23:45 IST\\n\\nIn Short\\n\\nFarmers\\' meeting in Haryana\\'s Kheri Chopta turns violent\\n\\nPolice fire tear gas, detain farmer leaders\\n\\nStone pelting by farmers escalates tension\\n\\nA meeting of farmers in Haryana\\'s Kheri Chopta turned violent on Friday as police fired tear gas and detained leaders. Thousands had gathered at the site after farmers\\' unions and khap panchayats

Finding Similarity between chunks of data

In [66]:
question = "What is fired at farmers?"

In [67]:
headers ={"Authorization": HuggingFaceApiKey}
API_URL = "https://api-inference.huggingface.co/models/sentence-transformers/all-MiniLM-L6-v2"
def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()
data = query(
    {
        "inputs": {
            "source_sentence": question,
            "sentences": chunks,
        }
    }
)

data_array = np.array(data)

max_indices = data_array.argsort()[-2:][::-1]

context = ""

for i in max_indices:
  context = context + chunks[i]

context

"[Document(page_content='Tear gas fired, stones thrown as farmers\\' meet in Haryana turns violent | Ground report\\n\\nThousands of farmers gathered in Haryana\\'s\\xa0Kheri Chopta to participate in a march towards the Khanauri border. The meeting turned violent after police fired tear gas and detained farmer leaders.\\n\\nListen to Story\\n\\nLive TV\\n\\nShare\\n\\nAdvertisement\\n\\nFarmers have been stationed at the Punjab-Haryana border as they march to Delhi to pressure the central government to fulfil their demands. (PTI photo)\\n\\nAnmol Nath Bali\\n\\nKheri Chopta,\\n\\nUPDATED: Feb 23, 2024 23:45 IST\\n\\nIn Short\\n\\nFarmers\\' meeting in Haryana\\'s Kheri Chopta turns violent\\n\\nPolice fire tear gas, detain farmer leaders\\n\\nStone pelting by farmers escalates tension\\n\\nA meeting of farmers in Haryana\\'s Kheri Chopta turned violent on Friday as police fired tear gas and detained leaders. Thousands had gathered at the site after farmers\\' unions and khap panchayats

Question Answering

In [68]:
API_URL = "https://api-inference.huggingface.co/models/deepset/roberta-base-squad2"
def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()
data = query(
    {
        "inputs": {
            "question": question,
            "context": context,
        }
    }
)
data

{'score': 0.5887683629989624, 'start': 1046, 'end': 1054, 'answer': 'tear gas'}

Summarization

In [69]:
API_URL = "https://api-inference.huggingface.co/models/facebook/bart-large-cnn"
def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()
data = query(
    {
        "inputs": data_,
        "parameters": {"do_sample": False},
    }
)
data

[{'summary_text': "Thousands of farmers gathered in Haryana's Kheri Chopta to participate in a march towards the Khanauri border. Meeting turned violent after police fired tear gas and detained farmer leaders. Bhartiya Kisan Union (Naujwan) President mourned the death of a farmer during clashes."}]